In [ ]:
import numpy as np, pandas as pd, scipy.stats as st

data = pd.read_csv('data.csv')
num_obvs = 30_000
# risks per sector
r = np.array([.295, .49, .41, .415, .338, .64, .403, .476])
#sec_loading maps sector to its risk, t is the threshold for defaults
data['sec_loading'], data['t'] = r[data['sector'].values], st.norm.ppf(data.p)
# 100k monte carlo simulations and len(r)+len(data) risk factors per one sample
num_dimensions = len(r) + len(data)
from scipy.stats import qmc

sobol_sampler = qmc.Sobol(d=num_dimensions, scramble=True)
quasi_random_samples = sobol_sampler.random_base2(m=int(np.log2(num_obvs)))
from scipy.stats import norm

factors = st.norm.ppf(quasi_random_samples)


def process_obs(obs):
    # m_factor, sec_factor, res_factor = obs[0], obs[1:1 + len(r)][data.sector.values], obs[len(r):]
    m_factor, sec_factor, res_factor = obs[0], obs[:len(r)][data.sector.values], obs[len(r):]

    ind = (
                  r[0] ** 0.5 * m_factor
                  + (data.sec_loading - r[0]) ** 0.5 * sec_factor
                  + (1 - data.sec_loading) ** 0.5 * res_factor
          ) < data.t
    loss = np.zeros((len(data),))
    stt = sobol_student.random(n=sum(ind)).flatten()
    stud = st.t.ppf(stt, 3)
    loss[ind] = data[ind].m + data[ind].d * stud
    return sum(loss)


from joblib import Parallel, delayed
import time
from scipy.stats import t

sample = []
sobol_student = qmc.Sobol(d=1, scramble=True)

sample = Parallel(n_jobs=-1)(delayed(process_obs)(obs) for obs in factors)
VaR = sorted([-s for s in sample])[int(0.01 * num_obvs)]
VaR